In [1]:
%pip install feast


[notice] A new release of pip is available: 24.1.1 -> 24.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


# Run a test client

> 🚀 This test is developer to work only with the default feature store generated by `feast init`. 
> 
> To test a custom feature store you need to run a custom test application, but still using the same client configuration that we've prepared.

## Apply the feature store definitions

The feature store cannot be initialized using remote services. 

We'll use the original `feature_store.yaml` from within a Kubernetes `Job` to run `feast apply`.

For the same reason, we also run an initial materialization from the `Job`, otherwise it would fail because of uninmplemented APIs in the remote servers, like [online_write_batch](https://github.com/feast-dev/feast/blob/4a6b663f80bc91d6de35ed2ec428d34811d17a18/sdk/python/feast/infra/online_stores/remote.py#L50).

First we create a `ConfigMap` holding the required code and configuration.

In [2]:
%env FEATURE_REPO_DIR=sample/feature_repo
!kubectl delete configmap sample-repo
!kubectl create configmap sample-repo --from-file=${FEATURE_REPO_DIR}/example_repo.py,${FEATURE_REPO_DIR}/feature_store.yaml
!echo
!echo "Inspect keys of sample-repo ConfigMap"
!kubectl get configmaps sample-repo -oyaml | yq '.data[] | key'

env: FEATURE_REPO_DIR=sample/feature_repo
Error from server (NotFound): configmaps "sample-repo" not found
configmap/sample-repo created

Inspect keys of sample-repo ConfigMap
example_repo.py
feature_store.yaml


Then we create the `Job` to apply the definitions, according to the [init-job.yaml](./init-job.yaml) manifest

In [3]:
!kubectl delete -f init-job.yaml
!kubectl apply -f init-job.yaml

Error from server (NotFound): error when deleting "init-job.yaml": jobs.batch "feast-apply-job" not found
job.batch/feast-apply-job created


Monitoring the log of the `Job`.

In [4]:
!INIT_JOB_POD=$(kubectl get pods -l job-name=feast-apply-job -oname) && kubectl wait --for=condition=podscheduled $INIT_JOB_POD --timeout=2m
!INIT_JOB_POD=$(kubectl get pods -l job-name=feast-apply-job -oname) && kubectl logs -f $INIT_JOB_POD


pod/feast-apply-job-tzscd condition met
Starting feast initialization job...
<jemalloc>: MADV_DONTNEED does not work (memset will be used instead)
<jemalloc>: (This is the expected behaviour if you are running under QEMU)
09/17/2024 11:18:10 AM feast.repo_config WARNING: The `path` of the `RegistryConfig` starts with a plain `postgresql` string. We are updating this to `postgresql+psycopg` to ensure that the `psycopg3` driver is used by `sqlalchemy`. If you want to use `psycopg2` pass `postgresql+psycopg2` explicitely to `path`. To silence this warning, pass `postgresql+psycopg` explicitely to `path`.
/usr/local/lib/python3.11/site-packages/feast/feature_store.py:590: RuntimeWarning: On demand feature view is an experimental feature. This API is stable, but the functionality does not scale well for offline retrieval
  warnings.warn(
Deploying infrastructure for driver_hourly_stats_fresh
Deploying infrastructure for driver_hourly_stats
<jemalloc>: MADV_DONTNEED does not work (memset wil

## Forwarding the feast service ports

To run the test client from the notebook, we need to forward the service ports to ports on the current host.

In [5]:
!kubectl get svc

NAME              TYPE        CLUSTER-IP     EXTERNAL-IP   PORT(S)          AGE
offline-server    ClusterIP   10.96.24.216   <none>        80/TCP           3m58s
online-server     ClusterIP   10.96.36.113   <none>        80/TCP           3m36s
postgres          NodePort    10.96.231.4    <none>        5432:30565/TCP   7m28s
registry-server   ClusterIP   10.96.128.48   <none>        80/TCP           4m25s


In [6]:
from src.utils import port_forward
registry_process = port_forward("registry-server", 8001)
offline_process = port_forward("offline-server", 8002)
online_process = port_forward("online-server", 8003)

Port-forwarding registry-server with process ID: 15094
Port-forwarding offline-server with process ID: 15095
Port-forwarding online-server with process ID: 15096


Forwarding from 127.0.0.1:8002 -> 8815
Forwarding from 127.0.0.1:8003 -> 6566
Forwarding from 127.0.0.1:8001 -> 6570
Forwarding from [::1]:8002 -> 8815
Forwarding from [::1]:8003 -> 6566
Forwarding from [::1]:8001 -> 6570


In [7]:
!ps -ef | grep port-forward

  501 15094 13456   0  1:18PM ??         0:00.06 kubectl port-forward service/registry-server 8001:80
  501 15095 13456   0  1:18PM ??         0:00.05 kubectl port-forward service/offline-server 8002:80
  501 15096 13456   0  1:18PM ??         0:00.06 kubectl port-forward service/online-server 8003:80
  501 15170 13456   0  1:18PM ttys051    0:00.14 /bin/zsh -c ps -ef | grep port-forward
  501 15173 15170   0  1:18PM ttys051    0:00.00 grep port-forward


## Client configuration
The client configuration is using only remote clients connected to the forwarded ports, from 8001 to 8003.

In [8]:
!cat client/feature_store.yaml

project: sample
registry:
  path: localhost:8001
  registry_type: remote
offline_store:
  host: localhost
  port: 8002
  type: remote
online_store:
  path: http://localhost:8003
  type: remote
entity_key_serialization_version: 2
auth:
  type: no_auth


## Install test code
First we copy the test code from `sample/feature_repo` to `client` folder.

In [9]:
!cp sample/feature_repo/test_workflow.py client
!ls client/*.py

client/__init__.py      client/test_workflow.py


We update the original test to comment the `apply`, `teardown` and `materialize-incremental` commands.

In [10]:
!sed -i.bk 's/subprocess.run/# subprocess.run/' client/test_workflow.py
!sed -i.bk 's/print("\\n--- Run feast/# print("\\n--- Run feast/' client/test_workflow.py
!sed -i.bk 's/store.materialize_incremental/# store.materialize_incremental/' client/test_workflow.py
!sed -i.bk 's/print("\\n--- Load features/# print("\\n--- Load features/' client/test_workflow.py
!diff client/test_workflow.py sample/feature_repo/test_workflow.py

12,13c12,13
<     # print("\n--- Run feast apply to setup feature store on Postgres ---")
<     # subprocess.run(["feast", "apply"])
---
>     print("\n--- Run feast apply to setup feature store on Postgres ---")
>     subprocess.run(["feast", "apply"])
21,22c21,22
<     # print("\n--- Load features into online store ---")
<     # store.materialize_incremental(end_date=datetime.now())
---
>     print("\n--- Load features into online store ---")
>     store.materialize_incremental(end_date=datetime.now())
56,57c56,57
<     # print("\n--- Run feast teardown ---")
<     # subprocess.run(["feast", "teardown"])
---
>     print("\n--- Run feast teardown ---")
>     subprocess.run(["feast", "teardown"])


Finally, we run the full test suite from the client folder.

In [11]:
!cd client && python test_workflow.py

Handling connection for 8001

--- Historical features for training ---
Handling connection for 8002
   driver_id     event_timestamp  ...  conv_rate_plus_val1  conv_rate_plus_val2
0       1001 2021-04-12 10:59:42  ...             1.302426            10.302426
1       1002 2021-04-12 08:12:10  ...             2.436384            20.436384
2       1003 2021-04-12 16:40:26  ...             3.954102            30.954102

[3 rows x 10 columns]

--- Historical features for batch scoring ---
Handling connection for 8002
   driver_id  ... conv_rate_plus_val2
0       1001  ...           10.798974
1       1002  ...           20.316096
2       1003  ...           30.202964

[3 rows x 10 columns]

--- Online features ---
Handling connection for 8003
acc_rate  :  [0.22748562693595886, 0.9316393733024597]
conv_rate_plus_val1  :  [1000.7989742159843, 1001.3160955905914]Handling connection for 8003

conv_rate_plus_val2  :  [2000.7989742159843, 2002.3160955905914]
driver_id  :  [1001, 1002]

--- Online

Note If you see the following error, it is likely due to the [issue #4392](https://github.com/feast-dev/feast/issues/4392):
Remote registry client does not map application errors:

```
Feature view driver_hourly_stats_fresh does not exist in project sample
```

## Terminate port forwarding

In [12]:
registry_process.terminate()
offline_process.terminate()
online_process.terminate()
!ps -ef | grep port-forward

  501 16434 13456   0  1:20PM ttys051    0:00.12 /bin/zsh -c ps -ef | grep port-forward
  501 16436 16434   0  1:20PM ttys051    0:00.00 grep port-forward
